In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from datasets import Dataset
import pandas as pd
import torch
import torch.nn.functional as F

# 1. 載入模型和 tokenizer
tokenizer = AutoTokenizer.from_pretrained("yentinglin/Llama-3-Taiwan-8B-Instruct", add_eos_token=True)
model = AutoModelForCausalLM.from_pretrained("yentinglin/Llama-3-Taiwan-8B-Instruct", low_cpu_mem_usage=True)

# 2. 配置 LoRA
lora_config = LoraConfig(
    r=64,  # 降維度
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],  # LoRA作用層
    lora_dropout=0.05,
    bias="none",  # 無 bias
    task_type="CAUSAL_LM",  # 用於自回歸模型
)

# 3. 準備模型訓練
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)  # 應用 LoRA 配置

# 4. 載入已訓練的 LoRA adapter
model.load_adapter("fine_tuned_model_0923_cus_good", adapter_name="adapter_model")

# 5. 設置最大輸入長度
tokenizer.model_max_length = 2200
model.config.max_position_embeddings = 2200
model.config.use_cache = False

# 6. 讀取 CSV 文件並打亂數據
data = pd.read_csv("comb_cus_Prompt_1025.csv")
data = data.sample(frac=1).reset_index(drop=True)

# 7. 將對話格式化
def format_conversation(row):
    chat = [
        {"role": "system", "content": row["Prompt"]},
        {"role": "user", "content": row["Input"] + " <|eot_id|>"},  # 添加自定義截停符
    ]
    conversation = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    return conversation

# 8. 對話格式化
data['conversation'] = data.apply(format_conversation, axis=1)

# 9. 將 pandas DataFrame 轉換為 Hugging Face Dataset 格式
dataset = Dataset.from_pandas(data[['conversation', 'Output']])

# 10. Tokenizer 處理
def tokenize_function(examples):
    conversations = [conv + " <|eot_id|>" for conv in examples['conversation']]
    outputs = [out + " <|eot_id|>" for out in examples['Output']]
    model_inputs = tokenizer(conversations, truncation=True, padding="max_length", max_length=2200)
    labels = tokenizer(outputs, truncation=True, padding="max_length", max_length=2200)
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

# 將數據集進行 tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 11. 訓練參數設定
training_args = TrainingArguments(
    output_dir="./training_results",
    evaluation_strategy="no",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1, # 增加梯度累計
    num_train_epochs=2,            # 增加訓練的epoch數
    weight_decay=0.01,             # 增加 weight decay
    logging_steps=1,               # 每10步記錄
    save_steps=500,                # 每500步儲存
    save_total_limit=2,            # 保留最新的兩個檔案
    learning_rate=1e-4,            # 調整學習率
    dataloader_num_workers=1,
    optim="adamw_torch",
)

class CustomTrainer(Trainer):
    def __init__(self, *args, l2_lambda=0.001, **kwargs):
        super().__init__(*args, **kwargs)
        self.l2_lambda = l2_lambda

    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        logits = outputs.logits

        # 採用 cross_entropy
        labels = inputs.get("labels")
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), reduction='mean')
        
        # L2正則
        l2_reg = sum(torch.norm(param, 2) for param in model.parameters())  # L2 參數

        # 總損失
        loss += self.l2_lambda * l2_reg
        
        # 梯度裁剪
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=12.0)

        return (loss, outputs) if return_outputs else loss

# 12. 構建 Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# 13. 開始訓練
trainer.train()

# 14. 儲存模型
trainer.save_model("./fine_tuned_model")

# 儲存 tokenizer
tokenizer.save_pretrained("./fine_tuned_model")